In [ ]:
import atomap.api as am
import numpy as np
import hyperspy.api as hs
import atomap.dummy_data as dummy_data

s = hs.load('/Users/usuario/Python PM Office/EPFL/20221028/L_A4-5_45/DF Stack_06 aligned sum using FFT spots Inverse.dm3')
s.plot()

In [ ]:
#get A positions
%matplotlib nbagg
A_positions = am.get_atom_positions(s, separation=25)
sublattice_A = am.Sublattice(A_positions, image=s.data, color='r')
atom_positions_add = am.add_atoms_with_gui(s, A_positions)

In [ ]:
atom_positions_select = am.select_atoms_with_gui(s, atom_positions_add)

In [ ]:
atom_positions_add = am.add_atoms_with_gui(s, atom_positions_select)

In [ ]:
sublattice_A = am.Sublattice(atom_positions_add, s)
sublattice_A.find_nearest_neighbors()
sublattice_A.refine_atom_positions_using_center_of_mass()
sublattice_A.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.4)
sublattice_A.construct_zone_axes()
sublattice_A.plot()
sublattice_A.plot_planes() #increase in case the columns are not very close in some regions check that one if it works atom_plane_tolerance=0.5
#sublattice_A._remove_bad_zone_vectors

In [ ]:
zone_axis_001 = sublattice_A.zones_axis_average_distances[1] #1 introduces vertical zone axes
B_positions = sublattice_A.find_missing_atoms_from_zone_vector(zone_axis_001)

In [ ]:
from atomap.tools import remove_atoms_from_image_using_2d_gaussian
image_without_A = remove_atoms_from_image_using_2d_gaussian(sublattice_A.image, sublattice_A)

In [ ]:
sublattice_B = am.Sublattice(B_positions, image_without_A, color='blue')
sublattice_B.construct_zone_axes()
sublattice_B.refine_atom_positions_using_center_of_mass()
sublattice_B.refine_atom_positions_using_2d_gaussian(percent_to_nn=0.2)

In [ ]:
atom_lattice = am.Atom_Lattice(image=s.data, name='test', sublattice_list=[sublattice_A, sublattice_B])
atom_lattice.plot()

In [ ]:
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_B = atom_lattice.sublattice_list[1]
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[0]
za1 = sublattice_A.zones_axis_average_distances[2]
sublattice_A.plot_planes(image=atom_lattice.image)# use this function to reveal the planes and select the appropriate ones. The number of the plane is indicated on the top
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_B)
vector_list = s_p.metadata.vector_list
x, y = [i[0] for i in vector_list], [i[1] for i in vector_list]
u, v = [i[2] for i in vector_list], [i[3] for i in vector_list]
sampling, units = 0.008902054, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='y', overlay=True, monitor_dpi=150)


In [ ]:
import temul.api as tml
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_B = atom_lattice.sublattice_list[1] ## for oxygen use sublattice_O 
sublattice_A.construct_zone_axes()
za0 = sublattice_A.zones_axis_average_distances[2]
za1 = sublattice_A.zones_axis_average_distances[0] ## For Oxygen use 1
s_p = sublattice_A.get_polarization_from_second_sublattice(za0, za1, sublattice_B) ## for oxygen use sublattice_O 
w=vector_list = s_p.metadata.vector_list

rows=15 #number of Ti rows.
columns=22  #number of Ti columns

# function to return the second element of the
def colsort():
    def sortSecond(val):
        return val[1] 
  
        #sort each column individualy. 
        # using the second key 
    
    d=rows
    p=0
    w.sort() #sort the list to the first element of each item which is the row.
    for i in range(columns): #number of columns
        s=w[p:d] #Select all the elemnts of the first column
        s.sort(key = sortSecond) #Sort the element from the top to bottom of the column
        w[p:d]=s # replace the elements of the list with the sorted ones
        p=p+rows
        d=d+rows
colsort()
w


In [ ]:
#**********Dipole filter**********
colsort()
x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

#print(len(u)) #check the length of the vector so as to create a list with the same size

elements=rows*columns # number of Oxygen atoms
resolution=0.006139348

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.5*u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [0.5*v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#print(w)  #use this function to check if the columns are sepparated correctly
#for diagonal multiplication a combination between bottom and side multiplication is needed. Specifically "for b in range(n+1)"

sampling, units =  resolution, 'nm' #image calibration
tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=sampling, units=units,unit_vector=False, save=None, scalebar=True, plot_style='vector', color='y', overlay=True, monitor_dpi=150)


In [ ]:
# filtered averaging
import statistics
import numpy
def mean(data):
    """Return the sample arithmetic mean of data."""
    n = len(data)
    if n < 1:
        raise ValueError('mean requires at least one data point')
    return sum(data)/n # in Python 2 use sum(data)/float(n)

def _ss(data):
    """Return sum of square deviations of sequence data."""
    c = mean(data)
    ss = sum((x-c)**2 for x in data)
    return ss
def stddev(data, ddof=0):
    """Calculates the population standard deviation
    by default; specify ddof=1 to compute the sample
    standard deviation."""
    n = len(data)
    if n < 2:
        raise ValueError('variance requires at least two data points')
    ss = _ss(data)
    pvar = ss/(n-ddof)
    return pvar**0.5

#Here I use the filter to get better averages values
colsort()        

x, y = [i[0] for i in w], [i[1] for i in w]
u, v = [i[2] for i in w], [i[3] for i in w]

#print(len(u)) #check the length of the vector so as to create a list with the same size

elements=1276 # number of Oxygen atoms
rows=28 # bumber of Oxygen rows
columns=43  #number of Oxygen columns
resolution=0.012364

#the list below is used to interate all over the vectors in the u and v lists.
list_n = [*range(elements)]

#bottom multiplication
w.append(w.pop(0)) # puts the first value of the list at the end of it. This process used so as to multiply the first vector with the one below.
u = [0.5*u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [0.5*v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#up multiplication
colsort()
for b in range(elements-1):
    w.append(w.pop(0))#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#side +1 column
colsort()
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side +2 column
for b in range(rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]

#side -1 column
colsort()
for b in range(elements-rows):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.15*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.15*i[3] for i,t in zip(w,list_n)]

#side -2 column
colsort()
for b in range(elements-rows*2):#this function moves the column to the end therefore we can make multiplication of the first columnm with the second
    w.append(w.pop(0))
u = [u[t]+0.05*i[2] for i,t in zip(w,list_n)]
v = [v[t]+0.05*i[3] for i,t in zip(w,list_n)]


#End of the filter



g=0;
c=0;
stdvp=list=[]  #list for stdv of horizontal displacement
stdvq=list=[]  #list for stdv of vertical displacement


print("        Δx                 Δy               STDVx              STDVy")
for q in range(rows): #this range is the number of rows
    t=q
    for i in range(columns): #this range is the number of columns
        b=u[t]
        z=v[t]
        c=c+b
        g=g+z
        t=t+rows     #this range is the number of rows
        stdvp.append(b*1000* resolution)
        stdvq.append(z*1000* resolution)
    
       # stdvq.append(z)
   ##  STDVp=numpy.std(arrp, axis=0)
   # arrq = numpy.array([q])
  #  e=numpy.std(arrq, axis=0)
    STDVp=stddev(stdvp)
    STDVq=stddev(stdvq)

    l=(1000*c*resolution)/columns  #horizontal x average value in pm
    a=(1000*g*resolution)/columns  #and vertical y average value in pm
    print(l, a, STDVp , STDVq ) 
    c=0;
    g=0;      
    stdvp=list=[]  #list for stdv of horizontal displacement
    stdvq=list=[]  #list for stdv of horizontal displacement
t=0    

In [ ]:
#for colored polarization map
>>> tml.plot_polarisation_vectors(x, y, u, v, image=atom_lattice.image,sampling=0.00613935,units='nm',
...                           unit_vector=False, save=None,
...                           plot_style='colormap', monitor_dpi=150,
...                           overlay=False, cmap='viridis')